In [1]:

!git clone https://github.com/adikan2k/Final-Project-Group-LexiCore.git


%cd /content/Final-Project-Group-LexiCore


!ls

Cloning into 'Final-Project-Group-LexiCore'...
remote: Enumerating objects: 470, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 470 (delta 45), reused 38 (delta 20), pack-reused 384 (from 2)
Receiving objects: 100% (470/470), 319.91 MiB | 20.91 MiB/s, done.
Resolving deltas: 100% (203/203), done.
/content/Final-Project-Group-LexiCore
 code  'Final Group Presentation'     LICENSE
 data  'Final Group Project Report'   README.md


In [2]:
!pip install lime shap sentence-transformers faiss-cpu streamlit rank-bm25 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 125.0 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import pickle
from pathlib import Path

import faiss
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression

from lime.lime_text import LimeTextExplainer
import shap
import matplotlib.pyplot as plt

ROOT = Path("/content/Final-Project-Group-LexiCore")

In [7]:
!mkdir -p /content/Final-Project-Group-LexiCore/models

In [9]:
%cd /content/Final-Project-Group-LexiCore

/content/Final-Project-Group-LexiCore


In [11]:
label_col = "venue"

In [12]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
import pickle
from pathlib import Path

df_path = "data/processed/cleaned_papers.parquet"
df = pd.read_parquet(df_path)

print("Loaded df with", len(df), "papers")
print("Columns:", list(df.columns))

label_col = "venue"

if label_col not in df.columns:
    raise ValueError(f"'{label_col}' not found. Available columns: {list(df.columns)}")

labels = df[label_col].astype(str)

print("Unique labels:", labels.unique())
print("Number of classes:", labels.nunique())


texts = df["title"].astype(str) + " " + df["original_abstract"].astype(str)

encoder = SentenceTransformer("all-MiniLM-L6-v2")

embs = encoder.encode(
    texts.tolist(),
    show_progress_bar=True
)

clf = LogisticRegression(max_iter=2000)
clf.fit(embs, labels)

print("Classifier trained successfully!")
print("Classes:", clf.classes_)


models_dir = Path("models")
models_dir.mkdir(exist_ok=True)

output_path = models_dir / "embedding_classifier.pkl"

with open(output_path, "wb") as f:
    pickle.dump({"classifier": clf, "classes": clf.classes_}, f)

print(f"Saved classifier to: {output_path}")

Loaded df with 500 papers
Columns: ['paper_id', 'title', 'authors', 'original_abstract', 'cleaned_text', 'language', 'sentences', 'num_sentences', 'tokens', 'processed_text', 'num_tokens', 'source', 'year', 'venue']
Unique labels: ['arXiv' 'Annual Meeting of the Association for Computational Linguistics'
 'Conference of the European Chapter of the Association for Computational Linguistics'
 'AAAI Conference on Artificial Intelligence' 'NLLP'
 'North American Chapter of the Association for Computational Linguistics'
 'Artificial Intelligence Review' 'Machine Learning with Applications'
 'Neural Information Processing Systems'
 'International Conference on Learning Representations'
 'Transactions of the Association for Computational Linguistics'
 'ACM Computing Surveys' 'SN Computer Science'
 'Social Network Analysis and Mining'
 'International Workshop on Semantic Evaluation'
 'IEEE Transactions on Knowledge and Data Engineering'
 'Conference on Empirical Methods in Natural Language Pro

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Classifier trained successfully!
Classes: ['' 'AAAI Conference on Artificial Intelligence' 'ACM Computing Surveys'
 'ACM Multimedia'
 'Annual Meeting of the Association for Computational Linguistics'
 'Applied Sciences' 'Artificial Intelligence Review' 'Bioinform.'
 'Computer Vision and Pattern Recognition'
 'Conference of the European Chapter of the Association for Computational Linguistics'
 'Conference on Computational Natural Language Learning'
 'Conference on Empirical Methods in Natural Language Processing'
 'European Conference on Computer Vision'
 'European Signal Processing Conference' 'Findings' 'IEEE Access'
 'IEEE International Conference on Acoustics, Speech, and Signal Processing'
 'IEEE International Conference on Computer Vision'
 'IEEE Transactions on Affective Computing'
 'IEEE Transactions on Knowledge and Data Engineering'
 'IEEE Transactions on Neural Networks and Learning Systems'
 'IEEE/ACM Transactions on Audio Speech and Language Processing'
 'Informatics' 'Int

In [14]:
!ls -l models

total 172
-rw-r--r-- 1 root root 175460 Dec  2 21:49 embedding_classifier.pkl


In [15]:
import pickle
from pathlib import Path
from sentence_transformers import SentenceTransformer

In [16]:
class EmbeddingClassifier:
    def __init__(self, encoder_model="all-MiniLM-L6-v2"):
        self.encoder = SentenceTransformer(encoder_model)
        self.classifier = None
        self.classes_ = None

    def attach(self, clf_obj):
        self.classifier = clf_obj["classifier"]
        self.classes_ = clf_obj["classes"]

    def predict_proba(self, text):
        emb = self.encoder.encode([text], show_progress_bar=False)
        return self.classifier.predict_proba(emb)

In [17]:
clf_path = Path("models/embedding_classifier.pkl")

if not clf_path.exists():
    raise FileNotFoundError("embedding_classifier.pkl not found under models/")

with open(clf_path, "rb") as f:
    clf_obj = pickle.load(f)

classifier_wrapper = EmbeddingClassifier()
classifier_wrapper.attach(clf_obj)

print("Loaded classifier with classes:", classifier_wrapper.classes_)

Loaded classifier with classes: ['' 'AAAI Conference on Artificial Intelligence' 'ACM Computing Surveys'
 'ACM Multimedia'
 'Annual Meeting of the Association for Computational Linguistics'
 'Applied Sciences' 'Artificial Intelligence Review' 'Bioinform.'
 'Computer Vision and Pattern Recognition'
 'Conference of the European Chapter of the Association for Computational Linguistics'
 'Conference on Computational Natural Language Learning'
 'Conference on Empirical Methods in Natural Language Processing'
 'European Conference on Computer Vision'
 'European Signal Processing Conference' 'Findings' 'IEEE Access'
 'IEEE International Conference on Acoustics, Speech, and Signal Processing'
 'IEEE International Conference on Computer Vision'
 'IEEE Transactions on Affective Computing'
 'IEEE Transactions on Knowledge and Data Engineering'
 'IEEE Transactions on Neural Networks and Learning Systems'
 'IEEE/ACM Transactions on Audio Speech and Language Processing'
 'Informatics' 'International

In [19]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import pickle
from pathlib import Path

from sentence_transformers import SentenceTransformer
from lime.lime_text import LimeTextExplainer


try:
    from run_pipeline import run_digest_query, digest_to_dataframe
    HAS_PIPELINE = True
except ImportError:
    HAS_PIPELINE = False


@st.cache_data
def load_corpus():
    df = pd.read_parquet("data/processed/cleaned_papers.parquet")
    return df

@st.cache_resource
def load_classifier():
    clf_path = Path("models/embedding_classifier.pkl")
    if not clf_path.exists():
        raise FileNotFoundError("models/embedding_classifier.pkl not found. "
                                "Make sure you committed the classifier file.")
    with open(clf_path, "rb") as f:
        clf_obj = pickle.load(f)
    return clf_obj

class EmbeddingClassifier:
    def __init__(self, encoder_model="all-MiniLM-L6-v2"):
        self.encoder = SentenceTransformer(encoder_model)
        self.classifier = None
        self.classes_ = None

    def attach(self, clf_obj):

        self.classifier = clf_obj["classifier"]
        self.classes_ = clf_obj["classes"]

    def predict_proba(self, texts):
        if isinstance(texts, str):
            texts = [texts]
        embs = self.encoder.encode(texts, show_progress_bar=False)
        return self.classifier.predict_proba(embs)


@st.cache_resource
def get_explainer(classifier_wrapper):
    return LimeTextExplainer(class_names=classifier_wrapper.classes_)



def simple_retrieval(df, query, top_k=5):
    """Very simple lexical retrieval using title+abstract contains query."""
    q = query.lower()
    combined = (df["title"].astype(str) + " " +
                df["original_abstract"].astype(str)).str.lower()
    mask = combined.str.contains(q)
    cand = df[mask].copy()
    if len(cand) == 0:
        return df.sample(min(top_k, len(df)))
    return cand.head(top_k)

def main():
    st.set_page_config(page_title="Scholarly Topic Navigator", layout="wide")
    st.title(" Scholarly Topic Navigator – Day 3 UI")
    st.markdown(
        "End-to-end **research digest** with summarization + classifier "
        "explainability (LIME) built in Streamlit."
    )


    with st.spinner("Loading corpus and classifier..."):
        df_corpus = load_corpus()
        clf_obj = load_classifier()
        classifier_wrapper = EmbeddingClassifier()
        classifier_wrapper.attach(clf_obj)
        explainer = get_explainer(classifier_wrapper)

    st.sidebar.header("Corpus Stats")
    st.sidebar.write(f"Total papers: `{len(df_corpus)}`")
    st.sidebar.write(f"Label column: `venue`")
    st.sidebar.write(f"Num classes: `{len(classifier_wrapper.classes_)}`")

    st.sidebar.markdown("---")
    st.sidebar.markdown("### How to use")
    st.sidebar.markdown(
        "1. Type a query\n"
        "2. See top matching papers\n"
        "3. Expand a paper to see summaries\n"
        "4. Click **Explain prediction** to view LIME explanation"
    )


    query = st.text_input(
        " Enter search query",
        value="transformer models",
        help="Try topics like 'graph neural networks', 'medical NLP', etc."
    )

    top_k = st.slider("Top K results", min_value=3, max_value=10, value=5, step=1)

    if st.button("Run search"):
        if not query.strip():
            st.warning("Please enter a query first.")
            st.stop()

        st.subheader(f"Results for: `{query}`")


        if HAS_PIPELINE:

            with st.spinner("Running full pipeline (hybrid retrieval + summarization)..."):
                digest = run_digest_query(query, top_k=top_k)
                df_digest = digest_to_dataframe(digest)
        else:

            with st.spinner("Running simple lexical retrieval..."):
                df_digest = simple_retrieval(df_corpus, query, top_k=top_k)

                if "one_sentence_summary" not in df_digest.columns:
                    df_digest["one_sentence_summary"] = (
                        df_digest["original_abstract"].str.split(".").str[0].fillna("")
                    )


        df_digest = df_digest.reset_index(drop=True)
        df_digest["rank"] = df_digest.index + 1


        for _, row in df_digest.iterrows():
            title = row.get("title", "Untitled paper")
            paper_id = row.get("paper_id", "unknown")
            venue = row.get("venue", "unknown")
            year = int(row.get("year", 0)) if "year" in row and pd.notna(row["year"]) else "N/A"

            with st.expander(f"#{row['rank']} – {title}  ({venue}, {year})"):
                abstract = row.get("original_abstract", "")
                st.markdown("**Abstract**")
                st.write(abstract)


                one_sent = row.get("one_sentence_summary", None)
                three_sent = row.get("three_sentence_summary", None)
                five_bullets = row.get("five_bullet_summary", None)

                if one_sent or three_sent or five_bullets:
                    st.markdown("---")
                    st.markdown("###  Summaries")

                    if one_sent:
                        st.markdown("**1-sentence summary**")
                        st.info(one_sent)

                    if three_sent:
                        st.markdown("**3-sentence summary**")
                        st.write(three_sent)

                    if five_bullets:
                        st.markdown("**5 key bullet insights**")
                        if isinstance(five_bullets, (list, tuple)):
                            for b in five_bullets:
                                st.markdown(f"- {b}")
                        else:
                            st.write(five_bullets)


                topic_label = row.get("topic_label", None)
                topic_id = row.get("topic_id", None)
                if topic_label or topic_id is not None:
                    st.markdown("---")
                    st.markdown("###  Topic assignment")
                    if topic_id is not None:
                        st.write(f"Topic ID: `{topic_id}`")
                    if topic_label:
                        st.write(f"Topic label: **{topic_label}**")


                st.markdown("---")
                st.markdown("###  Explain model prediction")

                explain_text = abstract or title
                if not explain_text:
                    st.info("No text available to explain.")
                else:
                    if st.button(f"Explain prediction for {paper_id}", key=f"explain-{paper_id}"):
                        with st.spinner("Running LIME explanation..."):
                            exp = explainer.explain_instance(
                                explain_text,
                                classifier_wrapper.predict_proba,
                                num_features=10
                            )
                            fig = exp.as_pyplot_figure()
                            st.pyplot(fig)
                            st.caption(
                                "Green = words that support the predicted venue; "
                                "red = words that push against it."
                            )


if __name__ == "__main__":
    main()

Overwriting app.py
